# Подготовка среды

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import pickle as pc
import numpy as np
import numpy

In [3]:
import os
os.chdir('/content/drive/Shared drives/hse_BERT/hse_Af_Tr_BERT')

In [4]:
!pip install transformers

In [5]:
!pip install razdel

In [7]:
import torch
import nltk
import re
import json
import time
import numpy as np
from transformers import BertTokenizer, BertModel

# OPTIONAL: if you want to have more information on what's happening, activate the logger as follows
import logging
#logging.basicConfig(level=logging.INFO)

import matplotlib.pyplot as plt
% matplotlib inline

from scipy.spatial.distance import cosine

# one dict

In [17]:
files = os.listdir("./embeddings/sent1/ready/")
all_emb = {}
for name in files:
    with open("./embeddings/sent1/ready/{}".format(name), 'rb') as f:
        data = pc.load(f)
        all_emb = {**all_emb, **data}
len(all_emb)

37800

In [18]:
with open("./embeddings/sent1/all_emc.pic", 'wb') as f:
    pc.dump(all_emb, f)

# read embeddings

In [8]:
files = os.listdir("./embeddings/sent1/bin")
cut_files = []
for i in files:
    if i.split('.')[-1] == 'pic':
        cut_files.append(i)
# cut_files

In [ ]:
# i = "1000_00024.pic"
# for i in cut_files:
#     with open("./embeddings/sent1/bin/{}".format(i), 'rb') as f:
#         data = pc.load(f)

##get count

In [9]:
%%time
from_files = {}
all_count = {}

for name in cut_files:
    with open("./embeddings/sent1/bin/{}".format(name), 'rb') as f:
        data = pc.load(f)
    for i in data.keys():
        if i not in all_count:
            all_count[i] = len(data[i])
            from_files[i] = [name]
        else:
            all_count[i] += len(data[i])
            from_files[i].append(name)
    # print(name)


CPU times: user 11min 13s, sys: 38.6 s, total: 11min 52s
Wall time: 15min 34s


In [10]:
out = list(zip(all_count.keys(), list(all_count.values())))
out.sort(key = lambda i: i[1], reverse=True)
out[:20]

[('как', 87089),
 ('что', 87073),
 ('его', 62700),
 ('the', 54443),
 ('в', 49243),
 ('и', 44503),
 ('это', 43184),
 ('для', 39351),
 ('так', 27947),
 ('все', 25799),
 ('или', 23508),
 ('с', 23001),
 ('только', 22587),
 ('and', 21309),
 ('жизни', 21113),
 ('она', 17073),
 ('том', 14964),
 ('этом', 14789),
 ('они', 14699),
 ('при', 14630)]

In [11]:
keys = list(all_count.keys())
k1 = []
k2 = []
k3 = []
kb = []
l = 0
for i in keys:
    l = len(i)
    if l == 1:
        k1.append(i)
    elif l == 2:
        k2.append(i)
    elif l == 3:
        k3.append(i)
    else:
        kb.append(i)
len(k1),len(k2),len(k3),len(kb)

(80, 549, 2092, 52515)

In [12]:
all_words = []
for i in k2 or i in k3:
    # if all_count[i] > 10:
        all_words.append(i)
len(all_words)

549

In [14]:
# qwe = list(range(225))[::-1]
len(all_words)// 100

5

In [13]:
def check_existing_emb(words):
    d = dict(out)
    fl = True
    for i in words:
        if i not in d:
            print("{} not in dictionary".format(i))
            fl = False
    return fl

def get_all_embeddingd(words):
    all_files = []
    embeddings = {}
    for w in words:
        embeddings[w] = []

    for i in words:
        all_files += from_files[i]

    for name in set(all_files):
        with open("./embeddings/sent1/bin/{}".format(name), 'rb') as f:
            data = pc.load(f)
        for w in words:
            if w in data:
                embeddings[w] += data[w]
    return embeddings

def sum_embeddings(embeddings):
    res = {}
    res_emb = {}
    for i in embeddings:
        res[i] = torch.empty(len(embeddings[i]), embeddings[i][0].size(0))
        for j in range(len(embeddings[i])):
            res[i][j] = embeddings[i][j]
        res_emb[i] = res[i].mean(dim = 0)
    return res_emb

In [16]:
%%time
st_time = time.time()
all_emb = {}
batch = 100
for b in range(len(all_words)// batch):
    t_time = time.time()
    words = all_words[b*batch:(b+1)*batch]
    emb = get_all_embeddingd(words)
    res_emb = sum_embeddings(emb)
    
    with open("./embeddings/sent1/ready/{}_{:05d}_2_3.pic".format(batch, b), 'wb') as f:
        pc.dump(res_emb, f)

    print("done: {} --- time: {}\t--- total: {}".format(b,time.time() - t_time, time.time() - st_time))

done: 0 --- time: 147.96394300460815	--- total: 147.96395349502563
done: 1 --- time: 105.01511549949646	--- total: 252.97934007644653
done: 2 --- time: 103.44571232795715	--- total: 356.4262218475342
done: 3 --- time: 101.93828797340393	--- total: 458.36605739593506
done: 4 --- time: 102.5061936378479	--- total: 560.8730847835541
CPU times: user 7min 44s, sys: 18.6 s, total: 8min 2s
Wall time: 9min 20s


In [ ]:
words

## get embeddings

In [ ]:
%%time
# words = ['мать', 'сын', 'отец', 'дочь', 'парень', 'девушка', 'машина', 'автомобиль', 'самолет', 'ребенок', 'королева', 'король', 'лед', 'тепло', 'вода']
words = ['россия','москв','франци','париж','украин','киев']

if check_existing_emb(words):
    emb = get_all_embeddingd(words)
    res_emb = sum_embeddings(emb)

In [ ]:
t1 = res_emb['франци'] - res_emb['париж'] + res_emb['киев']
t2 = res_emb['украин']
diff_bank = 1 - cosine(t1, t2)
print('Vector similarity for  *similar*  meanings:  %.2f' % diff_bank)

In [ ]:
mother = res_emb['мать']
father = res_emb['отец']
sun = res_emb['сын']
dot = res_emb['дочь']

In [ ]:
t1 = res_emb['девушка'] - res_emb['парень'] + res_emb['сын']
t2 = res_emb['дочь']
diff_bank = 1 - cosine(t1, t2)
print('Vector similarity for  *similar*  meanings:  %.2f' % diff_bank)

In [ ]:
t1 = res_emb['король'] - res_emb['парень'] + res_emb['девушка']
t2 = res_emb['королева']
diff_bank = 1 - cosine(t1, t2)
print('Vector similarity for  *similar*  meanings:  %.2f' % diff_bank)

In [ ]:
t1 = res_emb['король'] - res_emb['королева']
t2 = res_emb['девушка'] - res_emb['парень']
diff_bank = 1 - cosine(t1, t2)
print('Vector similarity for  *similar*  meanings:  %.2f' % diff_bank)

In [ ]:
t1 = res_emb['отец'] - res_emb['парень'] + res_emb['девушка']
t2 = res_emb['мать']
diff_bank = 1 - cosine(t1, t2)
print('Vector similarity for  *similar*  meanings:  %.2f' % diff_bank)

In [ ]:
t1 = res_emb['девушка']
t2 = res_emb['автомобиль']
diff_bank = 1 - cosine(t1, t2)
print('Vector similarity for  *similar*  meanings:  %.2f' % diff_bank)

In [ ]:
t1 = res_emb['машина']
t2 = res_emb['автомобиль']
diff_bank = 1 - cosine(t1, t2)
print('Vector similarity for  *similar*  meanings:  %.2f' % diff_bank)

In [ ]:
t1 = mother - father + sun
t2 = dot
diff_bank = 1 - cosine(t1, t2)
print('Vector similarity for  *similar*  meanings:  %.2f' % diff_bank)

In [ ]:
t1 = mother
t2 = dot
diff_bank = 1 - cosine(t1, t2)
print('Vector similarity for  *similar*  meanings:  %.2f' % diff_bank)

In [ ]:
t1 = father
t2 = sun
diff_bank = 1 - cosine(t1, t2)
print('Vector similarity for  *similar*  meanings:  %.2f' % diff_bank)

In [ ]:
t1 = sun
t2 = dot
diff_bank = 1 - cosine(t1, t2)
print('Vector similarity for  *similar*  meanings:  %.2f' % diff_bank)

In [ ]:
t1 = mother
t2 = father
diff_bank = 1 - cosine(t1, t2)
print('Vector similarity for  *similar*  meanings:  %.2f' % diff_bank)

In [ ]:
t1 = mother
t2 = sun
diff_bank = 1 - cosine(t1, t2)
print('Vector similarity for  *similar*  meanings:  %.2f' % diff_bank)

In [ ]:
t1 = father
t2 = dot
diff_bank = 1 - cosine(t1, t2)
print('Vector similarity for  *similar*  meanings:  %.2f' % diff_bank)

# short 

## imports

In [ ]:
import torch
import nltk
from transformers import BertTokenizer, BertModel

# OPTIONAL: if you want to have more information on what's happening, activate the logger as follows
import logging
#logging.basicConfig(level=logging.INFO)

import matplotlib.pyplot as plt
% matplotlib inline

from scipy.spatial.distance import cosine

# Load pre-trained model tokenizer (vocabulary)
tokenizer = BertTokenizer.from_pretrained('DeepPavlov/rubert-base-cased')

In [ ]:
# Load pre-trained model (weights)
model = BertModel.from_pretrained('DeepPavlov/rubert-base-cased',
                                  output_hidden_states = True, # Whether the model returns all hidden-states.
                                  )
# Put the model in "evaluation" mode, meaning feed-forward operation.
model.eval()

In [ ]:
from razdel import sentenize
# list(sentenize(text))

In [ ]:

nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
# sentence = """At eight o'clock on Thursday morning. Arthur didn't feel very good."""
tokens = nltk.word_tokenize(sentences[1])
tokens

In [ ]:
sentences = nltk.sent_tokenize(text)
sentences

In [ ]:
tagged = nltk.pos_tag(tokens)
tagged[0:6]

## work

In [ ]:
text = 'Замок стоял неприступный. Замок легко открылся.'
text2 = "At eight o'clock on Thursday morning. Arthur didn't feel very good."

In [ ]:
# tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# marked_text2 = "[CLS] " + "Работает по следующему принципу" + " [SEP]"
# tokenized_text2 = tokenizer.tokenize(marked_text2)
# tokenized_text2

In [ ]:
marked_text2 = text.lower()
marked_text = "[CLS] " + text + " [SEP]"

tokenized_text = tokenizer.tokenize(marked_text)
# tokenized_text = nltk.word_tokenize(marked_text)

# tokenized_text = ["[CLS]"] + tokenized_text + ["[SEP]"]

indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)
segments_ids = [1] * len(tokenized_text)

In [ ]:
tokenized_text

In [ ]:
# Convert inputs to PyTorch tensors
tokens_tensor = torch.tensor([indexed_tokens])
segments_tensors = torch.tensor([segments_ids])

In [ ]:
with torch.no_grad():
    outputs = model(tokens_tensor, segments_tensors)
    hidden_states = outputs[2]

In [ ]:
# token_i = 5
# layer_i = 5
# vec = hidden_states[layer_i][batch_i][token_i]

In [ ]:
# For the 5th token in our sentence, select its feature values from layer 5.
token_i = 5
layer_i = 5
vec = hidden_states[layer_i][batch_i][token_i]

# Plot the values as a histogram to show their distribution.
plt.figure(figsize=(10,10))
plt.hist(vec, bins=200)
plt.show()

In [ ]:
# Remove dimension 1, the "batches".
token_embeddings = torch.stack(hidden_states, dim=0)
token_embeddings = torch.squeeze(token_embeddings, dim=1).permute(1,0,2)

In [ ]:
token_vecs_sum = []
for token in token_embeddings:
    sum_vec = torch.sum(token[-4:], dim=0)
    token_vecs_sum.append(sum_vec)

print ('Shape is: %d x %d' % (len(token_vecs_sum), len(token_vecs_sum[0])))

In [ ]:
len(token_vecs_sum)

In [ ]:
for i, token_str in enumerate(tokenized_text):
  print (i, token_str)

In [ ]:
token_vecs = hidden_states[-2][0]
sentence_embedding = torch.mean(token_vecs, dim=0)

In [ ]:
token_vecs.size()

In [ ]:
diff_bank = 1 - cosine(token_vecs_sum[1], token_vecs_sum[7])
diff_bank1 = 1 - cosine(token_vecs[1], token_vecs[7])
# same_bank = 1 - cosine(token_vecs_sum[10], token_vecs_sum[6])

print('Vector similarity for  *similar*  meanings:  %.2f' % diff_bank)
print('Vector similarity for  *similar*  meanings:  %.2f' % diff_bank1)
# print('Vector similarity for *different* meanings:  %.2f' % diff_bank)

# Baseline



In [ ]:
import torch
from transformers import BertTokenizer, BertModel

# OPTIONAL: if you want to have more information on what's happening, activate the logger as follows
import logging
#logging.basicConfig(level=logging.INFO)

import matplotlib.pyplot as plt
% matplotlib inline

# Load pre-trained model tokenizer (vocabulary)
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [ ]:
text = "Here is the sentence I want embeddings for."
marked_text = "[CLS] " + text + " [SEP]"

# Tokenize our sentence with the BERT tokenizer.
tokenized_text = tokenizer.tokenize(marked_text)

# Print out the tokens.
print (tokenized_text)

In [ ]:
# Define a new example sentence with multiple meanings of the word "bank"
text = "After stealing money from the bank vault, the bank robber was seen " \
       "fishing on the Mississippi river bank."

# Add the special tokens.
marked_text = "[CLS] " + text + " [SEP]"

# Split the sentence into tokens.
tokenized_text = tokenizer.tokenize(marked_text)

# Map the token strings to their vocabulary indeces.
indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)

# Display the words with their indeces.
for tup in zip(tokenized_text, indexed_tokens):
    print('{:<12} {:>6,}'.format(tup[0], tup[1]))

In [ ]:
# Mark each of the 22 tokens as belonging to sentence "1".
segments_ids = [1] * len(tokenized_text)

In [ ]:
# Convert inputs to PyTorch tensors
tokens_tensor = torch.tensor([indexed_tokens])
segments_tensors = torch.tensor([segments_ids])

In [ ]:
# Load pre-trained model (weights)
model = BertModel.from_pretrained('bert-base-uncased',
                                  output_hidden_states = True, # Whether the model returns all hidden-states.
                                  )

# Put the model in "evaluation" mode, meaning feed-forward operation.
model.eval()

In [ ]:
# Run the text through BERT, and collect all of the hidden states produced
# from all 12 layers. 
with torch.no_grad():

    outputs = model(tokens_tensor, segments_tensors)

    # Evaluating the model will return a different number of objects based on 
    # how it's  configured in the `from_pretrained` call earlier. In this case, 
    # becase we set `output_hidden_states = True`, the third item will be the 
    # hidden states from all layers. See the documentation for more details:
    # https://huggingface.co/transformers/model_doc/bert.html#bertmodel
    hidden_states = outputs[2]

In [ ]:
# outputs.shape
hidden_states

In [ ]:
print ("Number of layers:", len(hidden_states), "  (initial embeddings + 12 BERT layers)")
layer_i = 0

print ("Number of batches:", len(hidden_states[layer_i]))
batch_i = 0

print ("Number of tokens:", len(hidden_states[layer_i][batch_i]))
token_i = 0

print ("Number of hidden units:", len(hidden_states[layer_i][batch_i][token_i]))

In [ ]:
# For the 5th token in our sentence, select its feature values from layer 5.
token_i = 5
layer_i = 5
vec = hidden_states[layer_i][batch_i][token_i]

# Plot the values as a histogram to show their distribution.
plt.figure(figsize=(10,10))
plt.hist(vec, bins=200)
plt.show()

In [ ]:
# Remove dimension 1, the "batches".

token_embeddings = torch.stack(hidden_states, dim=0)
token_embeddings = torch.squeeze(token_embeddings, dim=1)

token_embeddings.size()

# Swap dimensions 0 and 1.
token_embeddings = token_embeddings.permute(1,0,2)

token_embeddings.size()

In [ ]:
# Stores the token vectors, with shape [22 x 3,072]
token_vecs_cat = []

# `token_embeddings` is a [22 x 12 x 768] tensor.

# For each token in the sentence...
for token in token_embeddings:
    
    # `token` is a [12 x 768] tensor

    # Concatenate the vectors (that is, append them together) from the last 
    # four layers.
    # Each layer vector is 768 values, so `cat_vec` is length 3,072.
    cat_vec = torch.cat((token[-1], token[-2], token[-3], token[-4]), dim=0)
    
    # Use `cat_vec` to represent `token`.
    token_vecs_cat.append(cat_vec)

print ('Shape is: %d x %d' % (len(token_vecs_cat), len(token_vecs_cat[0])))

In [ ]:
# Stores the token vectors, with shape [22 x 768]
token_vecs_sum = []

# `token_embeddings` is a [22 x 12 x 768] tensor.

# For each token in the sentence...
for token in token_embeddings:

    # `token` is a [12 x 768] tensor

    # Sum the vectors from the last four layers.
    sum_vec = torch.sum(token[-4:], dim=0)
    
    # Use `sum_vec` to represent `token`.
    token_vecs_sum.append(sum_vec)

print ('Shape is: %d x %d' % (len(token_vecs_sum), len(token_vecs_sum[0])))

In [ ]:
# `hidden_states` has shape [13 x 1 x 22 x 768]

# `token_vecs` is a tensor with shape [22 x 768]
token_vecs = hidden_states[-2][0]

# Calculate the average of all 22 token vectors.
sentence_embedding = torch.mean(token_vecs, dim=0)

In [ ]:
from scipy.spatial.distance import cosine

# Calculate the cosine similarity between the word bank 
# in "bank robber" vs "river bank" (different meanings).
diff_bank = 1 - cosine(token_vecs_sum[10], token_vecs_sum[19])

# Calculate the cosine similarity between the word bank
# in "bank robber" vs "bank vault" (same meaning).
same_bank = 1 - cosine(token_vecs_sum[10], token_vecs_sum[6])

print('Vector similarity for  *similar*  meanings:  %.2f' % same_bank)
print('Vector similarity for *different* meanings:  %.2f' % diff_bank)